# Using 🤗 PEFT & bitsandbytes to finetune a LoRa checkpoint




In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3090 (UUID: GPU-53573cab-fc5d-e8af-4423-124f958899fb)
GPU 1: NVIDIA GeForce RTX 3090 (UUID: GPU-c0e0a88f-c719-20b0-0c44-8b697299c276)
GPU 2: NVIDIA GeForce RTX 3090 (UUID: GPU-0aa8929e-1774-1dc0-5325-0dce1165c78c)
GPU 3: NVIDIA GeForce RTX 3090 (UUID: GPU-9f605028-04a0-7456-bed1-8e4e06d9d44a)
GPU 4: NVIDIA GeForce RTX 3090 (UUID: GPU-31d09e65-80fe-5f2a-c34a-17da33412ae0)
GPU 5: NVIDIA GeForce RTX 3090 (UUID: GPU-d1a545c6-1038-527d-ec65-40f66caf2807)
GPU 6: NVIDIA GeForce RTX 3090 (UUID: GPU-7db6b504-9087-7bac-ba63-f957f018e494)
GPU 7: NVIDIA GeForce RTX 3090 (UUID: GPU-65bf5d44-7fd3-2034-09cf-4db9a2746b2a)


In [4]:
import transformers

### Setup the model

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

# device = "cuda:5"  # Replace "cuda:0" with the appropriate GPU device ID
# torch.cuda.set_device(device)

In [6]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
# import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model, TaskType

import os
os.environ["CUDA_VISIBLE_DEVICES"]="5"
import torch
import torch.nn as nn
import bitsandbytes as bnb
# import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForSeq2SeqLM.from_pretrained(
    "bigscience/mt0-xl",
    load_in_8bit=True,
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/mt0-xl")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/user/ksharma/miniconda3/envs/ks_tars_thesis did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No libcudart.so found! Install CUDA or the cudatoolkit package (anaconda)!
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MT5ForConditionalGeneration were not initialized from the model checkpoint at bigscience/mt0-xl and are newly initialized: ['encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Freezing the original weights


In [7]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Setting up the LoRa Adapters

In [8]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [9]:
from peft import LoraConfig, get_peft_model
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 9437184 || all params: 3752056832 || trainable%: 0.2515202840083207


## SJMM Data

In [10]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "5"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
# pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', 1000)
# pd.set_option('display.max_columns', None)
data_path="/home/user/ksharma/ks_thesis/ma-thesis/pipeline_test/data_files/df_2000_rows.csv"

# data_df = pd.read_csv(data_path)



# train_ratio = 0.9
# val_ratio = 0.1
# test_ratio = 0.1
from sklearn.model_selection import train_test_split
sjmm_data = pd.read_csv(data_path)
sjmm_data['input'] = sjmm_data['input'].str.rstrip(' ->')
fine_tuning_sjmm_data = sjmm_data[['input', 'output']]
fine_tuning_sjmm_data['ind_skill'] = fine_tuning_sjmm_data['output'].str.findall(r'<SoftSkill>(.*?)</SoftSkill>')
train_sjmm_data, test_sjmm_data = train_test_split(fine_tuning_sjmm_data, test_size=0.1, random_state=42)
# validation_sjmm_data, test_sjmm_data = train_test_split(test_sjmm_data, test_size=0.5, random_state=42)

In [11]:
# Create a new Dataset object
from datasets import Dataset, DatasetDict
train_sjmm_dataset = Dataset.from_dict(train_sjmm_data)

# Create a new DatasetDict object with the 'train' subset
train_sjmm_dataset_dict = DatasetDict({'train': train_sjmm_dataset})

In [12]:
train_sjmm_dataset_dict['train'][0]

{'input': '•  <SoftSkill_C>Hohes Kommunikations-</SoftSkill_C> und <SoftSkill>Verhandlungsgeschick</SoftSkill',
 'output': '<SoftSkill>Hohes Kommunikationsgeschick</SoftSkill> und <SoftSkill>Verhandlungsgeschick</SoftSkill>',
 'ind_skill': ['Hohes Kommunikationsgeschick', 'Verhandlungsgeschick']}

In [13]:
def merge_columns(example):
    example["prediction"] = "expand the text between <SoftSkill_C> and </SoftSkill_C> to make it complete. " + example["input"] + " == " + str(example["output"])
    return example

train_sjmm_dataset_dict['train'] = train_sjmm_dataset_dict['train'].map(merge_columns)
train_sjmm_dataset_dict['train']["prediction"][:5]

Map:   0%|          | 0/1773 [00:00<?, ? examples/s]

['expand the text between <SoftSkill_C> and </SoftSkill_C> to make it complete. •  <SoftSkill_C>Hohes Kommunikations-</SoftSkill_C> und <SoftSkill>Verhandlungsgeschick</SoftSkill == <SoftSkill>Hohes Kommunikationsgeschick</SoftSkill> und <SoftSkill>Verhandlungsgeschick</SoftSkill>',
 'expand the text between <SoftSkill_C> and </SoftSkill_C> to make it complete.  Bist du eine <SoftSkill_C>vielseitige</SoftSkill_C>, <SoftSkill_C>motivierte</SoftSkill_C> und <SoftSkill>lernorientierte Person</SoftSkill>, die <SoftSkill>Freude an der aktiven Gestaltung und Entwicklung anspruchsvoller Lösungen v.a. im Bereich der Digitalisierung hat</SoftSkill>? ==  Bist du eine <SoftSkill>vielseitige Person</SoftSkill>, <SoftSkill>motivierte Person</SoftSkill> und <SoftSkill>lernorientierte Person</SoftSkill>, die <SoftSkill>Freude an der aktiven Gestaltung und Entwicklung anspruchsvoller Lösungen v.a. im Bereich der Digitalisierung hat</SoftSkill>?',
 'expand the text between <SoftSkill_C> and </SoftSkill

In [14]:
# train_sjmm_dataset_dict
data_sjmm_dataset_dict = train_sjmm_dataset_dict.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/1773 [00:00<?, ? examples/s]

In [15]:
data_sjmm_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'ind_skill', 'prediction', 'input_ids', 'attention_mask'],
        num_rows: 1773
    })
})

In [16]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "5"

# device = "cuda:5"  # Replace "cuda:0" with the appropriate GPU device ID
# torch.cuda.set_device(device)

# import torch

# # Set the desired device
# device = "cuda:5"  # Replace "cuda:5" with the appropriate GPU device ID
# torch.cuda.set_device(device)

# # Move tensors in the dataset to the device
# for key in data_sjmm_dataset_dict['train']:
#     data_sjmm_dataset_dict['train'][key] = data_sjmm_dataset_dict['train'][key].to(device)


### Training

In [17]:

trainer = transformers.Trainer(
    model=model,
    # train_dataset=data['train'],
    train_dataset=data_sjmm_dataset_dict['train'],    
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=218,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs_mt0_XL_mt'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/home/user/ksharma/miniconda3/envs/ks_tars_thesis/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: kartikey-sharma (ma-thesis). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,0.396000
2,0.348600
3,0.388600
4,0.347800
5,0.303900
6,0.355800
7,0.394600
8,0.342900
9,0.403800
10,0.377400


TrainOutput(global_step=218, training_loss=0.0681661176072047, metrics={'train_runtime': 1034.6639, 'train_samples_per_second': 3.371, 'train_steps_per_second': 0.211, 'total_flos': 1.2851752387756032e+16, 'train_loss': 0.0681661176072047, 'epoch': 1.96})

## Share adapters on the 🤗 Hub

In [18]:
huggingface_token = "xxxxxxxxxxxxxxxxxx"
model.push_to_hub("Kartikey95/mt0_XL_with_prefix",
                  use_auth_token=huggingface_token,
                  commit_message="basic training",
                  private=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/37.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kartikey95/mt0_XL_with_prefix/commit/1f83273ae3e31885b72faa211eda089b5b5fecb9', commit_message='basic training', commit_description='', oid='1f83273ae3e31885b72faa211eda089b5b5fecb9', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

In [19]:
from huggingface_hub import login
import os

# access_token = os.environ["xxxxxxxxxxxxxxxxxx"]
login(token=huggingface_token)


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/user/ksharma/.cache/huggingface/token
Login successful


## Inference

In [24]:
from sklearn.model_selection import train_test_split
import pandas as pd
data_path="/home/user/ksharma/ks_thesis/ma-thesis/pipeline_test/data_files/df_2000_rows.csv"
sjmm_data = pd.read_csv(data_path)
sjmm_data['input'] = sjmm_data['input'].str.rstrip(' ->')
fine_tuning_sjmm_data = sjmm_data[['input', 'output']]
fine_tuning_sjmm_data['ind_skill'] = fine_tuning_sjmm_data['output'].str.findall(r'<SoftSkill>(.*?)</SoftSkill>')
pd.set_option('display.max_colwidth', 1000)
train_sjmm_data, test_sjmm_data = train_test_split(fine_tuning_sjmm_data, test_size=0.1, random_state=42)
test_sjmm_data

,input,output,ind_skill
752,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]"
765,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,"[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]"
1655,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikationsfähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","[psychische Gesundheit, physische Gesundheit, Kontaktfreudigkeit, gute Auffassungsgabe, Teamfähigkeit, Kommunikationsfähigkeit, praktisches Verständnis, manuelles Geschick]"
1252,Sie <SoftSkill_C>wertschätzen</SoftSkill_C> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.,Sie <SoftSkill>wertschätzen die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.,"[wertschätzen die Erfahrung der Betroffenen und deren Angehörigen, respektieren die Erfahrung der Betroffenen und deren Angehörigen]"
1102,"In unserer täglichen Arbeit legen wir grossen Wert darauf, <SoftSkill_C>wertschätzend</SoftSkill_C> und <SoftSkill>respektvoll miteinader zu kommunizieren</SoftSkill> und <SoftSkill>konstruktiv im Interesse einer hohen Patientenorientierung zusammen zu arbeiten</SoftSkill>.","In unserer täglichen Arbeit legen wir grossen Wert darauf, <SoftSkill>wertschätzend miteinader zu kommunizieren</SoftSkill> und <SoftSkill>respektvoll miteinader zu kommunizieren</SoftSkill> und <SoftSkill>konstruktiv im Interesse einer hohen Patientenorientierung zusammen zu arbeiten</SoftSkill>.","[wertschätzend miteinader zu kommunizieren, respektvoll miteinader zu kommunizieren, konstruktiv im Interesse einer hohen Patientenorientierung zusammen zu arbeiten]"
...,...,...,...
306,"Sie <SoftSkill>agieren in schwierigen Situationen ruhig</SoftSkill>, <SoftSkill_C>vertrauenswürdig</SoftSkill_C> und <SoftSkill_C>mit souveräner Umsicht</SoftSkill_C>.","Sie <SoftSkill>agieren in schwierigen Situationen ruhig</SoftSkill>, <SoftSkill>agieren vertrauenswürdig</SoftSkill> und <SoftSkill>agieren mit souveräner Umsicht</SoftSkill>.","[agieren in schwierigen Situationen ruhig, agieren vertrauenswürdig, agieren mit souveräner Umsicht]"
976,• <SoftSkill_C>Proaktives</SoftSkill_C> und <SoftSkill>vorausschauendes Denken</SoftSkill> und <SoftSkill_C>Handeln</SoftSkill_C,<SoftSkill>Proaktives Denken</SoftSkill> und <SoftSkill>vorausschauendes Denken</SoftSkill> und <SoftSkill>vorausschauendes Handeln</SoftSkill

In [28]:
# batch = tokenizer("<SoftSkill>Unternehmerisches Denken </SoftSkill> und <SoftSkillC>Handeln </SoftSkill_C> == ", return_tensors='pt')
import re
batch = tokenizer("expand the text between <SoftSkill_C> and </SoftSkill_C> to make it complete. Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>. == ", return_tensors='pt')


with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=580)

generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print('\n\n', generated_text)

match = re.search(r'\[(.*?)\]', generated_text)
extracted_text = match.group(1) if match else None
print('\n\n', extracted_text)




 expand the text between <SoftSkill_C> and </SoftSkill_C> to make it complete. Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>. == LINK_TEMPLATE


 None


In [26]:

import pandas as pd 
import re
from tqdm import tqdm

data_path="/home/user/ksharma/ks_thesis/ma-thesis/pipeline_test/data_files/MASTER_TEST_SET.csv"
results_mt0_XL = pd.read_csv(data_path)
results_mt0_XL = results_mt0_XL[results_mt0_XL['valid'] == 1]
results_mt0_XL['expected_skills'] = results_mt0_XL['expected_text'].str.findall(r'<SoftSkill>(.*?)</SoftSkill>')

# Create a new column 'generated_output' to store the generated output text
results_mt0_XL['generated_text'] = ''
results_mt0_XL['generated_skills_all'] = ''

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Tokenize the input text and add the generated output text to the DataFrame
for index, row in tqdm(results_mt0_XL.iterrows(), total=len(results_mt0_XL), desc="Generating Text"):

    input_text = row['input'] + " == "
    # expected_text = row['ind_skill']

    # Process the input text with the tokenizer to obtain the input tokens
    batch = tokenizer(input_text, return_tensors='pt')

    # Perform inference on the input tokens using your model
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, max_new_tokens=280)

    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    # Assign the generated output text to the corresponding row in the DataFrame
    results_mt0_XL.at[index, 'generated_text'] = generated_text

    gen_skills = re.findall(r'(?:SoftSkill>|SoftSkill_C>)(.*?)(?:/SoftSkill>|/SoftSkill_C>)', generated_text)
    results_mt0_XL.at[index, 'generated_skills_all'] = gen_skills

Generating Text:   0%|                                                                                                                                                                         | 0/195 [00:00<?, ?it/s]/home/user/ksharma/.local/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/user/ksharma/miniconda3/envs/ks_tars_thesis/lib/python3.9/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `inpu

KeyboardInterrupt: 

In [ ]:
results_mt0_XL.tail(5)

,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,generated_skills_all
193,306,"Sie <SoftSkill>agieren in schwierigen Situationen ruhig</SoftSkill>, <SoftSkill_C>vertrauenswürdig</SoftSkill_C> und <SoftSkill_C>mit souveräner Umsicht</SoftSkill_C>.","Sie <SoftSkill>agieren in schwierigen Situationen ruhig</SoftSkill>, <SoftSkill>agieren vertrauenswürdig</SoftSkill> und <SoftSkill>agieren mit souveräner Umsicht</SoftSkill>.",MWA,"[agieren in schwierigen Situationen ruhig, agieren vertrauenswürdig, agieren mit souveräner Umsicht]",1,"Sie <SoftSkill>agieren in schwierigen Situationen ruhig</SoftSkill>, <SoftSkill_C>vertrauenswürdig</SoftSkill_C> und <SoftSkill_C>mit souveräner Umsicht</SoftSkill_C>.","[agieren in schwierigen Situationen ruhig<, vertrauenswürdig<, mit souveräner Umsicht<]"
194,976,• <SoftSkill_C>Proaktives</SoftSkill_C> und <SoftSkill>vorausschauendes Denken</SoftSkill> und <SoftSkill_C>Handeln</SoftSkill_C,<SoftSkill>Proaktives Denken</SoftSkill> und <SoftSkill>vorausschauendes Denken</SoftSkill> und <SoftSkill>vorausschauendes Handeln</SoftSkill>,MDC,"[Proaktives Denken, vorausschauendes Denken, vorausschauendes Handeln]",1,• <SoftSkill_C>Proaktives</SoftSkill_C> und <SoftSkill>vorausschauendes Denken</SoftSkill> und <SoftSkill_C>Handeln</SoftSkill_C>,"[Proaktives<, vorausschauendes Denken<, Handeln<]"
195,1412,"<SoftSkill>Ausgeprägte Fähigkeiten in Konzeption, Analyse</SoftSkill>, <SoftSkill_C>Planung</SoftSkill_C> und <SoftSkill_C>Organisation</SoftSkill_C>; <SoftSkill>vernetztes Denken</SoftSkill> und <SoftSkill>kritisches Hinterfragen</SoftSkill","<SoftSkill>Ausgeprägte Fähigkeiten in Konzeption, Analyse</SoftSkill>, <SoftSkill>Ausgeprägte Fähigkeiten in Planung</SoftSkill> und <SoftSkill>Ausgeprägte Fähigkeiten in Organisation</SoftSkill>; <SoftSkill>vernetztes Denken</SoftSkill> und <SoftSkill>kritisches Hinterfragen</SoftSkill>",MC,"[Ausgeprägte Fähigkeiten in Konzeption, Analyse, Ausgeprägte Fähigkeiten in Planung, Ausgeprägte Fähigkeiten in Organisation, vernetztes Denken, kritisches Hinterfragen]",1,"<SoftSkill>Ausgeprägte Fähigkeiten in Konzeption, Analyse</SoftSkill>, <SoftSkill_C>Planung</SoftSkill_C> und <SoftSkill_C>Organisation</SoftSkill_C>; <SoftSkill>vernetztes Denken</SoftSkill> und <SoftSkill>kritisches Hinterfragen</SoftSkill","[Ausgeprägte Fähigkeiten in Konzeption, Analyse<, Planung<, Organisation<, vernetztes Denken<]"
196,1745,"• ƒ\n<SoftSkill_C>Bereitschaft</SoftSkill_C> und <SoftSkill>Fähigkeit, sich fachkompetent in der Projektarbeit zu vertiefen</SoftSkill","• <SoftSkill>Bereitschaft, sich fachkompetent in der Projektarbeit zu vertiefen</SoftSkill> und <SoftSkill>Fähigkeit, sich fachkompetent in der Projektarbeit zu vertiefen</SoftSkill>",MWA,"[Bereitschaft, sich fachkompetent in der Projektarbeit zu vertiefen, Fähigkeit, sich fachkompetent in der Projektarbeit zu vertiefen]",1,"• ƒ <SoftSkill_C>Bereitschaft</SoftSkill_C> und <SoftSkill>Fähigkeit, sich fachkompetent in der Projektarbeit zu vertiefen</SoftSkill",[Bereitschaft<]
197,100,"B und Hubstapler\n<SoftSkill>Prozessorientierte Denkweise</SoftSkill> und <SoftSkill>sehr gute IT-Affinität</SoftSkill>\n<SoftSkill_C>Selbständiges</SoftSkill_C> und <SoftSkill>sorgfältiges Arbeiten</SoftSkill> mit <SoftSkill>Eigeninitiative</SoftSkill>\n<SoftSkill>Bereitschaft, Samstag/Sonntag sowie an Feiertagen Arbeitseinsätze zu leisten</SoftSkill","B und Hubstapler\n<SoftSkill>Prozessorientierte Denkweise</SoftSkill> und <SoftSkill>sehr gute IT-Affinität</SoftSkill>\n<SoftSkill>Selbständiges Arbeiten</SoftSkill> und <SoftSkill>sorgfältiges Arbeiten</SoftSkill> mit <SoftSkill>Eigeninitiative</SoftSkill>\n<SoftSkill>Bereitschaft, Samstag/Sonntag sowie an Feiertagen Arbeitseinsätze zu leisten</SoftSkill>",OWA,"[Prozessorientierte Denkweise, sehr gute IT-Affinität, Selbständiges Arbeiten, sorgfältiges Arbeiten, Eigeninitiative, Bereitschaft, Samstag/Sonntag sowie an Feiertagen Arbeitseinsätze zu leisten]",1,"<SoftSkill>Pr

In [ ]:
# results_mt0_XL.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/bloom_peft_lora/bloom_7bn_results_15epochs.csv")
results_mt0_XL.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/mt0_xl/model_generation.csv")

: 